In [ ]:
%pwd
%cd ../..

In [ ]:
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import polars as pl

from deeplifting.problems import PROBLEMS_BY_NAME
from deeplifting.optimization import (
    run_deeplifting,
    run_differential_evolution,
    run_dual_annealing,
    run_ipopt,
    run_pygranso,
)
from deeplifting.utils import create_optimization_plot

In [ ]:
# fig = create_optimization_plot(problem_name, problem, results, colormap='autumn_r')

In [ ]:
# fig.savefig(
#     '/Users/ryandevera/data-science/umn_environments/Deeplifting'
#     f'/images/{problem_name}_surface_and_contour-autumn.png'
# )

In [ ]:
# Run all problems
problem_names = [
    'ackley',
    'bukin_n6',
    'cross_in_tray',
    'drop_wave',
    'eggholder',
    'griewank',
    'holder_table',
    'levy',
    'levy_n13',
    'rastrigin',
    'schaffer_n2',
    'schaffer_n4',
    'schwefel',
    'shubert',
    'ex8_1_1',
    # 'ex8_6_2',  # Need to fix threw an error
    'kriging_peaks_red010',
    'kriging_peaks_red020',
    'kriging_peaks_red030',
    'kriging_peaks_red050',
    'kriging_peaks_red100',
    'kriging_peaks_red200',
    'kriging_peaks_red500',
    'mathopt6',
    'quantum',
    'rosenbrock',
]

problem_performance = []
trials = 10

for problem_name in problem_names:
    problem = PROBLEMS_BY_NAME[problem_name]
    outputs = run_deeplifting(problem, trials=trials)
    results = outputs['final_results']

    # get the global minimum and the number of hits
    minimum_value = problem['global_minimum']
    hit = np.array([np.abs(found_min - minimum_value) for _, _, found_min in results])
    hits = np.where(hit <= 1e-4, 1, 0).mean()

    # Create a performance dataframe
    df = pl.DataFrame(
        {
            'problem_name': [problem_name] * trials,
            'algorithm': ['deeplifting'] * trials,
            'hits': hits,
        }
    )
    problem_performance.append(df)

In [ ]:
performance_df = pl.concat(problem_performance, how='vertical')

In [ ]:
performance_df = (
    performance_df
    .groupby(['problem_name'])
    .agg(pl.col('hits').mean())
    .rename({'hits': 'success_rate'})
    .sort('problem_name')
)

In [ ]:
performance_df